In [1]:
import pandas as pd
from collections import Counter
from langdetect import detect
import langdetect
import numpy as np
import importlib
import os
current_dir = os.getcwd()
%cd ..
import textmining.text_miner 
import textmining.loader  as loader
import textmining.topic_modeler as tm
import textmining.significance_testing as significance_testing
import textmining.utility as utility
import textmining.analysis as analysis
import textmining.cluster_analysis as cluster_analysis
import textmining.machine_learning as machine_learning
import textmining.news_editorials_experiments as experiment
import textmining.cross_models_significance as cross

importlib.reload(textmining.machine_learning)
importlib.reload(textmining.news_editorials_experiments)
importlib.reload(textmining.cross_models_significance)
importlib.reload(textmining.cluster_analysis)
importlib.reload(textmining.text_miner)

importlib.reload(textmining.text_miner)
importlib.reload(textmining.topic_modeler)
importlib.reload(textmining.analysis)
importlib.reload(textmining.loader)
os.chdir(current_dir)

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

C:\Users\elba_ro\Documents\projects\nlpcss2020-editorials-analysis


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elba_ro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elba_ro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elba_ro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
trait = 'neuroticism'

In [16]:
load = loader.loader()

"corpus" is set. It contains the 6000 annotation


In [17]:
personality_df = load.load_personality_traits()
personality_df.set_index('id', inplace=True)


In [18]:
# Traints
def set_avg_to_low(row):
    row[trait] = 'HIGH' if row[trait] == 'AVERAGE' else row[trait]
    return row
#personality_df = personality_df.apply(set_avg_to_low, axis=1)
personality_df[[trait]]
personality_df[trait].value_counts()

LOW        10
HIGH        9
AVERAGE     5
Name: neuroticism, dtype: int64

In [19]:
corpus = load.corpus
corpus = load.add_personality_label(personality_df[[trait]], trait)


"corpus" is set with personality: "a" , "b", ...


In [20]:
corpus[trait].value_counts()

LOW        2500
HIGH       2250
AVERAGE    1250
Name: neuroticism, dtype: int64

In [21]:
pd.crosstab(corpus[trait], corpus.effect_abstracted), print(''),print(''), pd.crosstab(corpus[trait], corpus.effect)

(effect_abstracted    1    2     3
 neuroticism                      
 AVERAGE            111  381   758
 HIGH               257  913  1080
 LOW                319  696  1485,
 None,
 None,
 effect        1    2    3     4    5
 neuroticism                         
 AVERAGE      16   95  381   568  190
 HIGH         52  205  913   625  455
 LOW          75  244  696  1007  478)

In [22]:
consc_maj_df = load.get_article_dfs_per_ideology(ideology = trait, include_content = True)

articles dataframe for ideology AVERAGE was created
The id of the df is the article id without txt
total con: 737
rounded TRAINGING data:  590
rounded Validation data:  0
rounded Test data:  147
total con: 1000
rounded TRAINGING data:  800
rounded Validation data:  0
rounded Test data:  200
"data_division" is set as dict with keys  dict_keys(['train', 'test'])
"corpus" is set with split_label: "train" "test"
"data_division" is set as dict with keys  dict_keys(['train', 'test'])
length of self.df:  737
length of self.data_division_df:  737
length of self.df:  737
articles dataframe for ideology HIGH was created
The id of the df is the article id without txt
total con: 979
rounded TRAINGING data:  783
rounded Validation data:  0
rounded Test data:  196
total con: 1000
rounded TRAINGING data:  800
rounded Validation data:  0
rounded Test data:  200
"data_division" is set as dict with keys  dict_keys(['train', 'test'])
"corpus" is set with split_label: "train" "test"
"data_division" is set

In [23]:
high_df = consc_maj_df['HIGH']
average_df = consc_maj_df['AVERAGE']
low_df = consc_maj_df['LOW']

In [ ]:
def check_if_all_0(df):
    df
    return  df[(df['challenging']== 1) & (df['no_effect']== 0) & (df['reinforcing']== 0)]

high_df['sum_']= high_df['challenging'] + high_df['no_effect'] +high_df['reinforcing']

low_df['sum_']= low_df['challenging'] + low_df['no_effect'] +low_df['reinforcing']
high_df['sum_'].value_counts(), low_df['sum_'].value_counts(),

In [24]:
print(low_df.majority.value_counts())

print()

print(average_df.majority.value_counts())
print()

print(high_df.majority.value_counts())


reinforcing    619
challenging    220
no_effect      140
Name: majority, dtype: int64

reinforcing    492
no_effect      142
challenging    103
Name: majority, dtype: int64

reinforcing    488
no_effect      381
challenging    110
Name: majority, dtype: int64


In [26]:
print(high_df.split_label.value_counts())
print()
print(average_df.split_label.value_counts())
print()
print(low_df.split_label.value_counts())
print()
print()
print(pd.crosstab(low_df.majority, low_df.split_label))
print()
print(pd.crosstab(average_df.majority, average_df.split_label))
print()
print(pd.crosstab(high_df.majority, high_df.split_label))

train    783
test     196
Name: split_label, dtype: int64

train    590
test     147
Name: split_label, dtype: int64

train    783
test     196
Name: split_label, dtype: int64


split_label  test  train
majority                
challenging    39    181
no_effect      36    104
reinforcing   121    498

split_label  test  train
majority                
challenging    23     80
no_effect      28    114
reinforcing    96    396

split_label  test  train
majority                
challenging    15     95
no_effect      70    311
reinforcing   111    377


In [ ]:
high_df.rename(columns = {'majority': trait+'_high_majority'}, inplace = True)
#average_df.rename(columns = {'majority': trait+'_average_majority'}, inplace = True)
low_df.rename(columns = {'majority': trait+'_low_majority'}, inplace = True)

In [ ]:
data = pd.read_json('../data/articles_with_adu_liwc_lexicons_content.json', orient='records')
data.set_index('idx', inplace=True)
print(len(data))
data = data.merge(high_df[[trait+'_high_majority']], how="inner",left_index=True, right_index=True )
#data = data.merge(average_df[[trait+'_average_majority']], how="inner",left_index=True, right_index=True )
data = data.merge(low_df[[trait+'_low_majority']], how="inner",left_index=True, right_index=True )

data.drop(['liberal_majority', 'conservative_majority'], axis=1, inplace=True)
data.to_csv('../data/'+trait+'_HL_all_features.csv')

In [ ]:
training_df, test_df = analysis.get_train_test(data)

# Training models

In [8]:
df = pd.read_csv('../data/'+trait+'_HAL_all_features.csv', index_col='idx')


In [ ]:

experiment.run_experiments(df, ideologies=[trait+'_high_majority', 
                                                          #trait+'_average_majority',
                                                         #trait+'_low_majority',
                                          ],
                                          filename="../out/style_content_models_results/"+trait+"_HL_experiments_sqrt",
                                          remove_outliers=True,
                                          normalize="sqrt")

running experiments for ideologies:  ['neuroticism_high_majority'] 
 remove_outliers:  True 
 normalize:  sqrt
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 434
getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_high_majority
+++++++++++++++++++++++++++++++++++++++++++++
('liwc',)
performing gridsearch
training and testing
macro-f1:  0.29 time(s):  6.981
-------------------------------------------
('nrc',)
performing gridsearch
training and testing
macro-f1:  0.32 time(s):  1.632
-------------------------------------------
('mpqa_arg',)
performing gridsearch


In [5]:
experiment.train_baseline(df, trait+'_high_majority')

removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 434
getting X y data...
Normalizing by using standard scaler...
Normalizing by using standard scaler...
normalized
end of get_x_y.


{'macro': 0.3,
 'micro': 0.34,
 'challenging': 0.1,
 'no_effect': 0.4,
 'reinforcing': 0.4}

In [ ]:
experiment.train_baseline(df, trait+'_average_majority')

In [6]:
experiment.train_baseline(df, trait+'_low_majority')

removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 434
getting X y data...
Normalizing by using standard scaler...
Normalizing by using standard scaler...
normalized
end of get_x_y.


{'macro': 0.31,
 'micro': 0.32,
 'challenging': 0.29,
 'no_effect': 0.27,
 'reinforcing': 0.37}

In [9]:
for l in ['low', 'high']:
    print(l)
    #print(results_minmax[ '{}_{}_majority'.format(trait, l)].macro.max() )
    #print(results_log[ '{}_{}_majority'.format(trait, l)].macro.max() )
    print(results_sqrt[ '{}_{}_majority'.format(trait, l)].macro.max() )
    #print(results_standard[ '{}_{}_majority'.format(trait, l)].macro.max() )


low


NameError: name 'results_sqrt' is not defined

In [10]:
level= 'high'
PATH_RESULT= '../out/style_content_models_results/{}_HAL_experiments_{}_{}_{}_majority.csv'

high_cross_models_sqrt_df    = cross.run_model_pairs_significance(PATH_RESULT.format(trait,'sqrt',trait, level), 
                                                                  df, '{}_{}_majority'.format(trait, level), 
                                                                  'sqrt'    , 'sqrt_{}_{}'.format(trait, level) 
                                                                 )



preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 435
getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_high_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('lemma',)) 2
('dummy',)
('lemma',)
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 435


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_high_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('nrc',)) 2
('dummy',)
('nrc',)
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 435


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_high_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('nrc', 'lemma')) 2
('dummy',)
('nrc', 'lemma')
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 435

...

c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_high_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('lemma',), ('nrc', 'lemma')) 2
('lemma',)
('nrc', 'lemma')
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 435


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_high_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('nrc',), ('nrc', 'lemma')) 2
('nrc',)
('nrc', 'lemma')


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [11]:
high_cross_models_sqrt_df

,model_pair,is_normal,stat,p_val,wilk_stat,wilk_p_val,significant,data
0,"((dummy,), (lemma,))",True,-1.007664,0.370623,4.0,0.345231,False,"{'('dummy',)': [0.19, 0.46, 0.2, 0.31, 0.23], ..."
1,"((dummy,), (nrc,))",True,-1.604733,0.183822,3.0,0.224916,False,"{'('dummy',)': [0.19, 0.46, 0.2, 0.31, 0.23], ..."
2,"((dummy,), (nrc, lemma))",True,-2.267891,0.085928,0.0,0.043114,False,"{'('dummy',)': [0.19, 0.46, 0.2, 0.31, 0.23], ..."
3,"((lemma,), (nrc,))",True,-0.770423,0.484029,5.0,0.500184,False,"{'('lemma',)': [0.28296703296703296, 0.3442802..."
4,"((lemma,), (nrc, lemma))",True,-2.000324,0.116074,3.0,0.224916,False,"{'('lemma',)': [0.28296703296703296, 0.3442802..."
5,"((nrc,), (nrc, lemma))",True,-1.388688,0.237252,3.0,0.224916,False,"{'('nrc',)': [0.337037037037037, 0.40566959921..."


In [12]:
level= 'average'

average_cross_models_sqrt_df    = cross.run_model_pairs_significance(PATH_RESULT.format(trait,'sqrt',trait, level), 
                                                                  df, '{}_{}_majority'.format(trait, level), 
                                                                  'sqrt'    , 'sqrt_{}_{}'.format(trait, level) 
                                                                 )

preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 435
getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_average_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('lemma',)) 2
('dummy',)
('lemma',)
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 435


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_average_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('mpqa_subjobg',)) 2
('dummy',)
('mpqa_subjobg',)
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 435


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_average_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('liwc', 'nrc', 'adu')) 2
('dummy',)
('liwc', 'nrc', 'adu')
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 435


...

c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_average_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('mpqa_subjobg',), ('mpqa_subjobg', 'adu', 'lemma')) 2
('mpqa_subjobg',)
('mpqa_subjobg', 'adu', 'lemma')
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 435


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_average_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('liwc', 'nrc', 'adu'), ('mpqa_subjobg', 'adu', 'lemma')) 2
('liwc', 'nrc', 'adu')
('mpqa_subjobg', 'adu', 'lemma')


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [13]:
average_cross_models_sqrt_df

,model_pair,is_normal,stat,p_val,wilk_stat,wilk_p_val,significant,data
0,"((dummy,), (lemma,))",True,-1.974696,0.119525,2.0,0.138011,False,"{'('dummy',)': [0.4, 0.22, 0.35, 0.2, 0.34], '..."
1,"((dummy,), (mpqa_subjobg,))",True,-0.358929,0.737792,6.0,0.685830,False,"{'('dummy',)': [0.4, 0.22, 0.35, 0.2, 0.34], '..."
2,"((dummy,), (liwc, nrc, adu))",True,-0.739998,0.500380,6.0,0.685830,False,"{'('dummy',)': [0.4, 0.22, 0.35, 0.2, 0.34], '..."
3,"((dummy,), (mpqa_subjobg, adu, lemma))",True,-1.754702,0.154167,2.0,0.138011,False,"{'('dummy',)': [0.4, 0.22, 0.35, 0.2, 0.34], '..."
4,"((lemma,), (mpqa_subjobg,))",True,0.820298,0.458114,5.0,0.500184,False,"{'('lemma',)': [0.3120879120879121, 0.39661572..."
5,"((lemma,), (liwc, nrc, adu))",True,1.694459,0.165426,2.0,0.138011,False,"{'('lemma',)': [0.3120879120879121, 0.39661572..."
6,"((lemma,), (mpqa_subjobg, adu, lemma))",True,0.201426,0.850194,5.0,0.500184,False,"{'('lemma',)': [0.3120879120879121, 0.39661572..."
7,"((mpqa_subjobg,), (liwc, nrc, adu))",True,-0.089308,0.933130,7.0,0.892738,False,"{'('mpqa_subjobg',)': [0.2108974358974359, 0.5..."
8,"((mpqa_subjobg,), (mpqa_subjobg, adu, lemma))",True,-0.860991,0.437794,5.0,0.500184,False,"{'('mpqa_subjobg',)': [0.2108974358974359, 0.5..."
9,"((liwc, nrc, adu), (mpqa_subjobg, adu, lemma))",True,-1.307117,0.261249,3.0,0.224916,False,"{'('liwc', 'nrc', 'adu')': [0.2771672771672771..."


In [14]:
level= 'low'

low_cross_models_sqrt_df    = cross.run_model_pairs_significance(PATH_RESULT.format(trait,'sqrt',trait, level), 
                                                                  df, '{}_{}_majority'.format(trait, level), 
                                                                  'sqrt'    , 'sqrt_{}_{}'.format(trait, level) 
                                                                 )



preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 435
getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_low_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('lemma',)) 2
('dummy',)
('lemma',)
preprocessing data...
removing outliers by clipping values...

c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")



getting only numeric features from the training set...
There are 428  numeric features out of 435
getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_low_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('liwc', 'mpqa_arg', 'mpqa_subjobg', 'adu')) 2
('dummy',)
('liwc', 'mpqa_arg', 'mpqa_subjobg', 'adu')
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 435


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_low_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('dummy',), ('liwc', 'mpqa_arg', 'lemma')) 2
('dummy',)
('liwc', 'mpqa_arg', 'lemma')


...

c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")



getting only numeric features from the training set...
There are 428  numeric features out of 435
getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_low_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('mpqa_arg', 'lemma'), ('mpqa_subjobg', 'lemma')) 2
('mpqa_arg', 'lemma')
('mpqa_subjobg', 'lemma')
preprocessing data...
removing outliers by clipping values...
getting only numeric features from the training set...
There are 428  numeric features out of 435

c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")



getting X y data...
Normalizing by using sqrt scaler...
normalized
end of get_x_y.
END of preprocessing
neuroticism_low_majority
+++++++++++++++++++++++++++++++++++++++++++++
(('liwc', 'mpqa_arg', 'adu', 'lemma'), ('mpqa_subjobg', 'lemma')) 2
('liwc', 'mpqa_arg', 'adu', 'lemma')
('mpqa_subjobg', 'lemma')


c:\programdata\miniconda3\lib\site-packages\scipy\stats\morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [15]:
low_cross_models_sqrt_df

,model_pair,is_normal,stat,p_val,wilk_stat,wilk_p_val,significant,data
0,"((dummy,), (lemma,))",True,0.852921,0.441765,4.0,0.345231,False,"{'('dummy',)': [0.22, 0.43, 0.47, 0.45, 0.35],..."
1,"((dummy,), (liwc, mpqa_arg, mpqa_subjobg, adu))",True,0.207749,0.845574,7.0,0.892738,False,"{'('dummy',)': [0.22, 0.43, 0.47, 0.45, 0.35],..."
2,"((dummy,), (liwc, mpqa_arg, lemma))",True,0.401250,0.708748,5.0,0.500184,False,"{'('dummy',)': [0.22, 0.43, 0.47, 0.45, 0.35],..."
3,"((dummy,), (mpqa_arg, mpqa_subjobg, lemma))",True,0.405668,0.705748,6.0,0.685830,False,"{'('dummy',)': [0.22, 0.43, 0.47, 0.45, 0.35],..."
4,"((dummy,), (mpqa_arg, lemma))",True,0.539757,0.618010,5.0,0.500184,False,"{'('dummy',)': [0.22, 0.43, 0.47, 0.45, 0.35],..."
5,"((dummy,), (liwc, mpqa_arg, adu, lemma))",True,0.419534,0.696378,5.0,0.500184,False,"{'('dummy',)': [0.22, 0.43, 0.47, 0.45, 0.35],..."
6,"((dummy,), (mpqa_subjobg, lemma))",True,0.600445,0.580572,5.0,0.500184,False,"{'('dummy',)': [0.22, 0.43, 0.47, 0.45, 0.35],..."
7,"((lemma,), (liwc, mpqa_arg, mpqa_subjobg, adu))",True,-0.372058,0.728722,6.0,0.685830,False,"{'('lemma',)': [0.2735042735042735, 0.33333333..."
8,"((lemma,), (liwc, mpqa_arg, lemma))",True,-0.511403,0.636005,6.0,0.685830,False,"{'('lemma',)': [0.2735042735042735, 0.33333333..."
9,"((lemma,), (mpqa_arg, mpqa_subjobg, lemma))",True,-0.189506,0.858924,7.0,0.892738,False,"{'('lemma',)': [0.2735042735042735, 0.33333333..."


In [ ]:
PATH_RESULT= '../out/style_content_models_results/{}_HAL_experiments_{}_{}_{}_majority.csv'

#singles_low, low_res = cross.get_top_features_from_path(PATH_RESULT.format(trait,'sqrt',trait, 'low'))
#singles_avg, avg_res = cross.get_top_features_from_path(PATH_RESULT.format(trait,'sqrt',trait, 'average'))
singles_high, high_res = cross.get_top_features_from_path(PATH_RESULT.format(trait,'sqrt',trait, 'high'))



In [ ]:
low_res['content+style'][['features','macro', 'ideology']]
low_res['style'][['features','macro', 'ideology']]

low_res['content'][['features','macro', 'ideology']]
#singles_low[(singles_low['single'] == True )& (singles_low['type'] == 'style')][['features','macro', 'ideology']]

In [ ]:
#avg_res['content+style'][['features','macro', 'ideology']]
#avg_res['style'][['features','macro', 'ideology']]
#avg_res['content'][['features','macro', 'ideology']]
singles_avg[(singles_avg['single'] == True )& (singles_avg['type'] == 'style')][['features','macro', 'ideology']]

In [ ]:
high_res['content+style'][['features','macro', 'ideology']]
#high_res['style'][['features','macro', 'ideology']]
#high_res['content'][['features','macro', 'ideology']]
#singles_high[(singles_high['single'] == True )& (singles_high['type'] == 'style')][['features','macro', 'ideology']]